In [20]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma

from langchain.llms import GooglePalm
from langchain.chains import RetrievalQA

from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.tools import BaseTool
from langchain.chains import LLMMathChain
from langchain.utilities import SerpAPIWrapper

import os

In [16]:
from dotenv import dotenv_values
ENV = dotenv_values("../.env")

## Vector Store

In [5]:
def get_vector_store_from_docs():
   docs = []
   for pdf in os.listdir("../pdf_EN"):
      loader = PyPDFLoader(f"../pdf_EN/{pdf}")
      docs.extend(loader.load())

   text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=30)
   chunked_docs = text_splitter.split_documents(docs)
   
   vector_db = Chroma.from_documents(chunked_docs, embedding=HuggingFaceEmbeddings(), persist_directory="../vector_store_EN")
   vector_db.persist()

   return vector_db   

In [9]:
vector_store_db = get_vector_store_from_docs()

## Model: Agent + QA Retrievals

In [29]:
llm = GooglePalm(temperature=0, google_api_key=ENV["PALM_API_KEY"])

In [30]:
rag = RetrievalQA.from_chain_type(
   llm=llm,
   chain_type="stuff",
   retriever=vector_store_db.as_retriever(),
)

In [35]:
search = SerpAPIWrapper(serpapi_api_key=ENV["SERP_API_KEY"])

tools = [
    Tool(
        name="ReACT",
        func=rag.run,
        description="useful for when you need to answer questions about synergizing reasoning and acting in language models.",
        return_direct=True
    ),
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    )
]

In [36]:
agent = initialize_agent(
   tools=tools,
   llm=llm,
   agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
   verbose=True
)

In [33]:
agent.run("What did biden say about ketanji brown jackson in the state of the union address?")



> Entering new AgentExecutor chain...
I need to know what biden said about ketanji brown jackson in the state of the union address
Action: Search
Action Input: biden said about ketanji brown jackson in the state of the union address
Observation: Brilliant. A brilliant legal mind with deep knowledge of the law. And a judicial temperament — which was equally important, in my view — that's calm and in command. And a humility that allows so many Americans to see themselves in Ketanji Brown Jackson.
Thought:I now know the final answer
Final Answer: Brilliant. A brilliant legal mind with deep knowledge of the law. And a judicial temperament — which was equally important, in my view — that's calm and in command. And a humility that allows so many Americans to see themselves in Ketanji Brown Jackson.

> Finished chain.


"Brilliant. A brilliant legal mind with deep knowledge of the law. And a judicial temperament — which was equally important, in my view — that's calm and in command. And a humility that allows so many Americans to see themselves in Ketanji Brown Jackson."

In [37]:
agent.run("What is synergizing reasoning and acting in language models?")



> Entering new AgentExecutor chain...
I need to use ReACT to answer this question
Action: ReACT
Action Input: synergizing reasoning and acting in language models
Observation: The paper proposes a method called ReAct, which stands for REasoning and ACTing in Language Models. ReAct allows LLMs to generate both reasoning traces and task-specific actions in an interleaved manner, allowing for greater synergy between the two. Reasoning traces help the model induce, track, and update action plans as well as handle exceptions, while actions allow it to interface with and gather additional information from external sources such as knowledge bases or environments. ReAct is evaluated on a diverse set of language and decision making tasks, and it demonstrates its effectiveness over state-of-the-art baselines in addition to improved human interpretability and trustworthiness.


> Finished chain.


'The paper proposes a method called ReAct, which stands for REasoning and ACTing in Language Models. ReAct allows LLMs to generate both reasoning traces and task-specific actions in an interleaved manner, allowing for greater synergy between the two. Reasoning traces help the model induce, track, and update action plans as well as handle exceptions, while actions allow it to interface with and gather additional information from external sources such as knowledge bases or environments. ReAct is evaluated on a diverse set of language and decision making tasks, and it demonstrates its effectiveness over state-of-the-art baselines in addition to improved human interpretability and trustworthiness.'